# Explainable Sentiment Analysis - Interactive Demo

This notebook demonstrates the explainable sentiment analysis system with interactive examples.


In [ ]:
import sys
import os
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), 'src'))

import torch
from transformers import AutoTokenizer
from model import InterpretableSentimentClassifier, SentimentModelWrapper
from explainability import (
    IntegratedGradientsExplainer, SHAPLikeExplainer,
    RationaleGenerator, CounterfactualGenerator
)
import matplotlib.pyplot as plt
import numpy as np


## Load Model


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_name = 'roberta-base'

print(f"Loading model: {model_name} on {device}")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = InterpretableSentimentClassifier(
    model_name=model_name,
    num_labels=3
)
model.to(device)
model.eval()

wrapper = SentimentModelWrapper(model, tokenizer, device=device)
print("Model loaded!")


## Example: Basic Prediction and Explanation


In [ ]:
text = "I absolutely love this product! It's amazing and works perfectly."

# Get prediction
prediction = wrapper.predict_text(text)
print(f"Text: {text}")
print(f"\nPredicted Label: {prediction['label']}")
print(f"Confidence: {prediction['confidence']:.2%}")
print(f"\nProbabilities:")
for label, prob in prediction['probabilities'].items():
    print(f"  {label}: {prob:.2%}")

# Get attributions
ig_explainer = IntegratedGradientsExplainer(model, tokenizer, device=device)
attributions = ig_explainer.get_attributions(text)

# Display top contributing tokens
token_scores = list(zip(attributions['tokens'], attributions['attributions']))
token_scores.sort(key=lambda x: abs(x[1]), reverse=True)

print("\nTop Contributing Tokens:")
for token, score in token_scores[:10]:
    direction = "↑" if score > 0 else "↓"
    print(f"{direction} '{token}': {score:+.3f}")

# Generate rationale
rationale_gen = RationaleGenerator()
rationale = rationale_gen.generate(
    prediction['label'],
    prediction['confidence'],
    attributions,
    prediction['probabilities']
)

print(f"\nRationale: {rationale}")
